In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import xgboost as xgb
from time import time
import gc
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [2]:
#path = '/Users/804357/Desktop/MyFiles/Learn/LOM/Data/'
path = '/Users/Vishy/Files/AVDatahack/LOM/Data/' 
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')
camp = pd.read_csv(path+'campaign_data.csv')

train_df = pd.merge(train, camp, on='campaign_id')
test_df = pd.merge(test, camp, on='campaign_id')
print('Shape of the train data is', train_df.shape)
print('Shape of the test data is', test_df.shape)

Shape of the train data is (1023191, 14)
Shape of the test data is (773858, 12)


In [4]:
train_df.columns

Index(['id', 'user_id', 'campaign_id', 'send_date', 'is_open', 'is_click',
       'communication_type', 'total_links', 'no_of_internal_links',
       'no_of_images', 'no_of_sections', 'email_body', 'subject', 'email_url'],
      dtype='object')

In [11]:
def overlap(list1,list2):
    listC = []
    for element in list1:
        if element in list2:
            listC.append(element)
    return len(listC)        
        
def distinct(list1,list2):
    listD = []
    for element in list1:
        if element not in list2:
            listD.append(element)
    return len(listD)        

In [17]:
print('The Number of unique users in Train and Test Data respectively are', train_df.user_id.nunique(), 
      test_df.user_id.nunique())
print('---------------------------------------------------------------')
print('The number of users from Train and in Test are:',len(set(train_df.user_id.unique())&set(test_df.user_id.unique())))
print('---------------------------------------------------------------')
print('The number of users from Train and not in Test are:',len(set(train_df.user_id.unique())^set(test_df.user_id.unique())))

The Number of unique users in Train and Test Data respectively are 168236 198219
---------------------------------------------------------------
The number of users from Train and in Test are: 145737
---------------------------------------------------------------
The number of users from Train and not in Test are: 74981


In [19]:
print('The Number of unique campaign in Train and Test Data respectively are', train_df.campaign_id.nunique(), 
      test_df.campaign_id.nunique())
print('----------------------------------------')
print('The number of campaigns from Train and in Test are:', len(set(train_df.campaign_id.unique())&set(train_df.campaign_id.unique())))
print('---------------------------------------------------------------')
print('The number of campaigns from Train and not in Test are:', len(set(train_df.campaign_id.unique())^set(test_df.campaign_id.unique())))

The Number of unique campaign in Train and Test Data respectively are 26 26
----------------------------------------
The number of campaigns from Train and in Test are: 26
---------------------------------------------------------------
The number of campaigns from Train and not in Test are: 52


In [25]:
print(train_df.campaign_id.nunique())
print(train_df.campaign_id.unique())
print(test_df.campaign_id.nunique())
print(test_df.campaign_id.unique())

26
[42 52 33 44 29 49 30 54 36 53 32 48 50 41 47 39 34 46 35 31 37 38 45 51
 40 43]
26
[63 56 57 71 76 74 67 75 64 69 78 79 61 58 77 60 80 65 70 55 68 73 59 66
 72 62]


In [26]:
print(train_df.communication_type.nunique())
print(train_df.communication_type.unique())
print(test_df.communication_type.nunique())
print(test_df.communication_type.unique())

7
['Newsletter' 'Others' 'Upcoming Events' 'Conference' 'Corporate'
 'Hackathon' 'Webinar']
4
['Newsletter' 'Upcoming Events' 'Hackathon' 'Corporate']


In [27]:
train_df.is_open.value_counts()

0    920401
1    102790
Name: is_open, dtype: int64

In [28]:
train_df.is_click.value_counts()

0    1010409
1      12782
Name: is_click, dtype: int64